In [2]:
import ast
import astmonkey_O



In [3]:

class nodeMutate(ast.NodeTransformer):

    changed_Name = False

    def visit(self, node):
        """Visit a node."""
        terminate = False
        method = 'visit_' + node.__class__.__name__
        if ("Name" == node.__class__.__name__): 
            if self.changed_Name == True:
                return node, terminate
            terminate = True
            self.changed_Name = True
        visitor = getattr(self, method, self.generic_visit)
        return visitor(node), terminate
    
    def generic_visit(self, node):
        for field, old_value in ast.iter_fields(node): # iterate the fields to get the node_type that you want to change
            # print("field is %s and old_value is %s" % (field, old_value))
            if isinstance(old_value, list):
                new_values = []
                for value in old_value:
                    if isinstance(value, ast.AST):  # change only one at a time
                        value, t = self.visit(value)
                        if t: 
                            print("jyjyrryh"); 
                            # return node
                        if value is None:
                            continue
                        elif not isinstance(value, ast.AST):
                            new_values.extend(value)
                            continue
                    new_values.append(value)
                old_value[:] = new_values
            elif isinstance(old_value, ast.AST):
                new_node, t = self.visit(old_value)
                if new_node is None:
                    delattr(node, field)
                else:
                    setattr(node, field, new_node)
                    
        return node

    def visit_Num(self, node):
        return ast.copy_location(ast.Constant(n=node.n+1), node)
    
    def visit_Constant(self, node):
        return ast.copy_location(ast.Constant(n=node.n+1), node)

    def visit_Str(self, node):
        return ast.copy_location(ast.Constant(s=node.s+"1"), node)

    def visit_Name(self, node):
        return ast.copy_location(ast.Name(id=node.id+"1", ctx=node.ctx), node)

    def visit_List(self, node):
        return ast.copy_location(ast.List(elts=[self.visit(x) for x in node.elts], ctx=node.ctx), node)

    def visit_Tuple(self, node):
        return ast.copy_location(ast.Tuple(elts=[self.visit(x) for x in node.elts], ctx=node.ctx), node)

    def visit_Set(self, node):
        return ast.copy_location(ast.Set(elts=[self.visit(x) for x in node.elts], ctx=node.ctx), node)

    def visit_Dict(self, node):
        return ast.copy_location(ast.Dict(keys=[self.visit(x) for x in node.keys], values=[self.visit(x) for x in node.values]), node)

    def visit_Attribute(self, node):
        return ast.copy_location(ast.Attribute(value=self.visit(node.value), attr=node.attr+"1", ctx=node.ctx), node)

    def visit_Subscript(self, node):
        return ast.copy_location(ast.Subscript(value=self.visit(node.value), slice=self.visit(node.slice), ctx=node.ctx), node)

    def visit_Index(self, node):
        return ast.copy_location(ast.Index(value=self.visit(node.value)), node)

    def visit_Slice(self, node):
        return ast.copy_location(ast.Slice(lower=self.visit(node.lower), upper=self.visit(node.upper), step=self.visit(node.step)), node)

    def visit_ExtSlice(self, node):
        return ast.copy_location(ast.ExtSlice(dims=[self.visit(x) for x in node.dims]), node)

    def visit_IfExp(self, node):
        return ast.copy_location(ast.IfExp(test=self.visit(node.test), body=self.visit(node.body), orelse=self.visit(node.orelse)), node)

    def visit_Compare(self, node):
        return ast.copy_location(ast.Compare(left=self.visit(node.left), ops=node.ops, comparators=[self.visit(x) for x in node.comparators]), node)

    def visit_Call(self, node):
        return ast.copy_location(ast.Call(func=self.visit(node.func), args=[self.visit(x) for x in node.args], keywords=[self.visit(x) for x in node.keywords]), node)
    
    def visit_Name(self, node):
        return ast.Subscript(
            value= ast.Name(id='data', ctx=ast.Load()),
            slice= ast.Constant(value=node.id),
            ctx=node.ctx
        )

    def visit_Module(self, node):
        return ast.copy_location(ast.Module(body=[self.visit(x) for x in node.body]), node)

try:
    code = ast.parse("a = 1\nb = 2\nc = a + 3") # this code has the following attributes: Num (1, 2, 3), Name (a, b), Assign, BinOp ( a+ 3)
    # mutator = nodeMutate()
    c = nodeMutate().generic_visit(code)
    print(astmonkey_O.to_source(c))
    # print(astmonkey_O.to_source(c))
    # mutator.visit(code)
    # code = mutator.visit(code)
    # print(astmonkey_O.to_source(code))
    
    # for node in ast.walk(code):
    #     print(node)
except Exception as e:
    print(e)
    # print("Error in parsing")

jyjyrryh
data['a'] = 2
b = 3
c = a + 4


In [4]:
from typing import List
import itertools

def findTypeLine(line):
    tokensArrSplit = [',', '(', ')',":"] # may result in empty element in list
    def flatten(lst):
        return list(itertools.chain.from_iterable(flatten(item) if isinstance(item, list) else [item] for item in lst))

    def splitMe(code: List | str, tokensArrSplit: List, index:int):
        if (index == len(tokensArrSplit)): return code
        listCodeTokens = []
        if (type(code) is list):
            for (i, c) in enumerate(code):
                if (c == ""): continue
                splitted = c.split(tokensArrSplit[index])
                # print(splitted)
                if (not splitted): continue
                listCodeTokens.append(splitted)
            listCodeTokens  = flatten(listCodeTokens)
            return splitMe(listCodeTokens, tokensArrSplit, index + 1)
        else:
            listCodeTokens = code.split(tokensArrSplit[index])
            return splitMe(listCodeTokens, tokensArrSplit, index + 1)
    
    testFunc = "def func(y , x , z): return u"
    print(splitMe(testFunc, tokensArrSplit, 0))

findTypeLine("p")



['def func', 'y ', ' x ', ' z', '', ' return u']


In [5]:

s = astmonkey_O.to_source(code)
print(s)
a  = (ast.parse(s))
print(ast.dump(a, indent=4))

data['a'] = 2
b = 3
c = a + 4
Module(
    body=[
        Assign(
            targets=[
                Subscript(
                    value=Name(id='data', ctx=Load()),
                    slice=Constant(value='a'),
                    ctx=Store())],
            value=Constant(value=2)),
        Assign(
            targets=[
                Name(id='b', ctx=Store())],
            value=Constant(value=3)),
        Assign(
            targets=[
                Name(id='c', ctx=Store())],
            value=BinOp(
                left=Name(id='a', ctx=Load()),
                op=Add(),
                right=Constant(value=4)))],
    type_ignores=[])


In [6]:
# every visitor class has its implementation of visitor and its visit_name;
# this way each class has its own implementation of the visitor and the visit_name

class BinOpMutate(ast.NodeTransformer):
    def __init__(self, target_node_lineno):
        self.target_node_lineno = target_node_lineno
        self.changedAnOperator = False

    def visit_BinOp(self, node):
        if node.lineno == self.target_node_lineno and not self.changedAnOperator:  # However if two operations are on the same line, it will change both
            self.changedAnOperator = True
            return ast.BinOp(left=self.visit(node.left), op=ast.Sub(), right=self.visit(node.right))
        else:
            # If it's not the target node, continue visiting other nodes without modifications
            return self.generic_visit(node)

code = ast.parse("a = 1\nb = 2\nc = a + 3;m = 5 + 4") # this code has the following attributes: Num (1, 2, 3), Name (a, b), Assign, BinOp ( a+ 3)

res = BinOpMutate(target_node_lineno=3).visit(code)
print(astmonkey_O.to_source(res))

a = 1
b = 2
c = a - 3
m = 5 + 4


In [7]:
%load_ext autoreload
%autoreload 2

import mutationClasses
import ast
import astmonkey_O

In [13]:
codeString = """
if len(S)<26:
    for i in m[::-1]:
        a = 1 + 2
        if i not in S:
            S.append(i)
            break
    print(''.join(S))
else:
    print(''.join(S))
"""
code = ast.parse(codeString)
codeStringSplitted = codeString.split("\n")
testParse = codeStringSplitted[3]
testAst = ast.parse(testParse.lstrip())
print(ast.dump(testAst, indent=4))

# print(code)
# binOpMutate = mutationClasses.BinOpMutate(target_node_lineno=3, code=code)
code =  mutationClasses.BinOpMutate(target_node_lineno=4, code=code).visitC()

# res =  mutationClasses.BinOpMutate(target_node_lineno=4).visitC(code)
print(astmonkey_O.to_source(code))

Module(
    body=[
        Assign(
            targets=[
                Name(id='a', ctx=Store())],
            value=BinOp(
                left=Constant(value=1),
                op=Add(),
                right=Constant(value=2)))],
    type_ignores=[])

if len(S) < 26:
    for i in m[::-1]:
        a = 1 - 2
        if i not in S:
            S.append(i)
            break
    print(''.join(S))
else:
    print(''.join(S))
